In [1]:
!uv sync

Resolved 139 packages in 243ms
Audited 135 packages in 1ms


In [10]:
from time import time
import math
import h3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
import seaborn as sns

In [ ]:
%pwd
%cd ../

c:\Users\Диана\Documents\GitHub\BigData-project


In [14]:
summary_shape = 0
try:
    path = './data/transactions.parquet'
    best_engine = 'pyarrow'
    transactions = pd.read_parquet(path, engine=best_engine)
    transactions.name = 'transactions'
    transactions[['lat', 'lon']] = transactions['h3_09'].apply(h3.cell_to_latlng).to_list()
    print("Transactions shape: ", transactions.shape)
    summary_shape += transactions.shape[0]

    path = './data/moscow.parquet'
    best_engine = 'pyarrow'
    moscow = pd.read_parquet(path, engine=best_engine)
    moscow.name = 'moscow'
    moscow = moscow[moscow['tags'].notna()]
    print("Moscow shape: ", moscow.shape)
    summary_shape += moscow.shape[0]

    path = './data/target.parquet'
    best_engine = 'fastparquet'
    target = pd.read_parquet(path, engine=best_engine)
    target.name = 'target'
    target[['lat', 'lon']] = target['h3_09'].apply(h3.cell_to_latlng).to_list()
    print("Target shape: ", target.shape)
    summary_shape += target.shape[0]
except FileNotFoundError:
    print("Error: File transactions.parquet wasn't found.  Put the proper path.")

Transactions shape:  (4151096, 13)
Moscow shape:  (571107, 4)
Target shape:  (157806, 4)


In [15]:
moscow

,id,tags,lat,lon
0,1000,"[(comment, ООО ""АСП-ГРУПП"", ИНН 7735108290, до...",55.995115,37.207532
1,26609007,"[(ele, 190), (iata, SVO), (icao, UUEE), (name,...",55.987408,37.420944
2,26999673,"[(alt_name, Москва-Октябрьская), (esr:user, 06...",55.778834,37.653721
16,26999981,"[(name, Подрезково), (name:en, Podrezkovo), (p...",55.941962,37.334361
18,26999985,"[(esr:user, 060406), (express:user, 2005090), ...",55.949653,37.299001
...,...,...,...,...
8867761,11574072808,"[(addr:flats, 1-18), (entrance, staircase), (r...",55.788764,37.639026
8867766,11574072813,"[(barrier, bollard)]",55.788792,37.639404
8867769,11574072816,"[(barrier, lift_gate), (bicycle, yes), (foot, ...",55.788636,37.639270
8867774,11574347530,"[(amenity, library), (contact:telegram, https:...",55.770631,37.675718


In [8]:
print(transactions.isnull().sum())

h3_09                   0
customer_id             0
datetime_id             0
count                   0
sum                     0
avg                     0
min                     0
max                     0
std               2399551
count_distinct          0
mcc_code                0
lat                     0
lon                     0
dtype: int64


For std column it is ok, according to data desciption we need to change nan to 0 if count_distinct is equal to 1.

In [9]:
print(target.isnull().sum())

h3_09          0
customer_id    0
lat            0
lon            0
dtype: int64


In [10]:
print(moscow.isnull().sum())

id      0
tags    0
lat     0
lon     0
dtype: int64


In [16]:
def open_lst(path):
    with open(path, "r") as f:
        df = pd.DataFrame({'h3_09': [x.strip() for x in f.readlines()]})
        df[['lat', 'lon']] = df['h3_09'].apply(h3.cell_to_latlng).to_list()
    return df

hexses_data = open_lst('./data/hexses_data.lst')
hexses_data.name = 'hexses_data'
print("hexses_data shape: ", hexses_data.shape)
summary_shape += hexses_data.shape[0]
hexses_target = open_lst('./data/hexses_target.lst')
hexses_target.name = 'hexses_target'
print("hexses_target shape: ", hexses_target.shape)
summary_shape += hexses_target.shape[0]
print("Summary shape: ", summary_shape)

hexses_data shape:  (8154, 3)
hexses_target shape:  (1657, 3)
Summary shape:  4889820


In [17]:
hexses_data.duplicated().any()

np.False_

In [17]:
hexses_target.duplicated().any()

np.False_

In [12]:
print(hexses_data.isnull().sum())

h3_09    0
lat      0
lon      0
dtype: int64


In [13]:
print(hexses_target.isnull().sum())

h3_09    0
lat      0
lon      0
dtype: int64


In [18]:
#create a new dataset - hexses, that contains data from both datasets hexses_data and hexses_target
hexses = pd.concat([hexses_data, hexses_target]).drop_duplicates().reset_index(drop=True)
hexses.name = 'hexses'
print(hexses.shape[0])

8157


In [16]:
hexses

,h3_09,lat,lon
0,89118180927ffff,55.245280,37.361687
1,89118180d27ffff,55.247340,37.326151
2,891181820abffff,55.215345,37.163678
3,891181840a7ffff,55.322955,37.399937
4,891181844c3ffff,55.309257,37.350979
...,...,...,...
8152,8911aa7b6dbffff,55.645766,37.486863
8153,8911aaccacfffff,55.755457,36.935436
8154,8911aa4c2a3ffff,55.567224,37.463483
8155,8911aa63473ffff,55.796397,37.647286


In [21]:
hexses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8157 entries, 0 to 8156
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   h3_09   8157 non-null   object 
 1   lat     8157 non-null   float64
 2   lon     8157 non-null   float64
dtypes: float64(2), object(1)
memory usage: 191.3+ KB


In [19]:
def check_values_inclusion(source_col: pd.Series, target_col: pd.Series) -> bool:
    
    non_nan_check = source_col.dropna().isin(target_col.dropna()).all()
    
    nan_check = (
        not source_col.hasnans 
        or (source_col.hasnans and target_col.isna().any())
    )
    
    return non_nan_check and nan_check

result = check_values_inclusion(transactions["h3_09"], hexses["h3_09"])
print(f"All values of h3_09 from transactions are in hexses: {result}")

All values of h3_09 from transactions are in hexses: True


In [20]:
result = check_values_inclusion(target["h3_09"], hexses["h3_09"])
print(f"All values of h3_09 from target are in hexses: {result}")

All values of h3_09 from target are in hexses: True


In [20]:
moscow['h3_09'] =  moscow[['lat','lon']].apply(lambda row: h3.latlng_to_cell(row['lat'], row['lon'], 9), axis=1) 

In [21]:
result = check_values_inclusion(moscow["h3_09"], hexses["h3_09"])
print(f"All values of h3_09 from moscow are in hexses: {result}")

All values of h3_09 from moscow are in hexses: False
